# <div align="center">Ocorrência de Câncer de Mama - UCI Machine Learning Repository</div>

<br>
<div align="center"><img src="https://i.imgur.com/71iwWHI.png"></div>
<hr>

## Introdução

Esta análise irá explorar o campo das ciências biológicas, especificamente na **ocorrência de câncer de mama**, por meio dos dados disponibilizados pelo **Centro de Machine Learning e Sistemas Inteligentes da Universidade da California (UCI)**, localizada nos Estados Unidos. 

O **câncer de mama** se caracteriza por ser o tipo **mais incidente** em mulheres no mundo totalizando acima de **2.5 milhões de casos** representando **25% de novos casos** de novas ocorrências de câncer em mulheres segundo o *IARC* (2020). No Brasil não é diferente sendo **o tipo mais indicente em mulheres em todas as regiões do país**. Segundo estimativas do *INCA*, em 2022, é previsto em torno de **685 mil casos** da doença por aqui.


A pesquisa foi realizada pelo **Instituto de Oncologia do Centro Médico Universitário** de **Ljubljana** na ocasião da antiga Iugoslávia, hoje, capital da **Eslovênia**.

O dataset é composto por **286 ocorrências** de câncer de mama se dividindo em duas classes:
- **Recorrente**: possuindo 201 instâncias
- **Não recorrente**: 85 instâncias

E são descritos por 9 atributos, a seguir:
- **Idade**
- **Menopausa**: pré ou pós
- **Tamanho do Tumor**: em milímetros
- **Número de Nódulos**: 0 a 39
- **Ocorrência de Metastase**
- **Grau de Malignidade**: tendência de tumores se tornarem progressivamente piores. Grau 1 (menor) a 3 (maior)
- **Seio**: esquerdo ou direito
- **Quadrante de ocorrência**
- **Irradiação**: ocorrência ou não de tratamento por radioterapia

<br>

**Fonte**
<div style="font-size: 12px">
Dua, D. and Graff, C. (2019). <a href="http://archive.ics.uci.edu/ml">UCI Machine Learning Repository</a>. Irvine, CA: University of California, School of Information and Computer Science.
</div>

<hr>


## Análise

### Tipos de Variáveis

Na Estatística Descritiva, em um modo sucinto, as variáveis podem ser classificadas em **qualitativas** ou **quantitativas**

As **qualitativas ou categóricas** expressam atributos que podem ser separados em diferentes categorias que se diferem por caraterísticas não numéricas. Podem ser:

1. **Nominais** em que não há uma ordenação específica: 
    - Gênero de uma pessoa (cisgênero/ transgênero/ não-binário)
    - Cor dos olhos (negros/ azuis/ verdes/ etc)
    - Categorias de filmes (terros/drama/ação/etc)
    - Etc

2. **Ordinais** quando existe certa ordem em seus valores
    - Nível de resfriamento de um congelador: baixo, médio, alto
    - Nível de aquecimento de um chuveiro: frio, morno e médio
    - Etc


As **quantitativas** são aquelas que podem ser medidas ou contabilizadas em termos de números. Elas podem ser 

1. **Discretas** quando resultam de um conjunto finito (enumerável, contável) de valores possíveis. Representam ***contagens***
    - Total de livros de uma biblioteca
    - Total de pessoas um evento
    - Quantidade de votos em uma eleição
    - Carros em um estacionamento

2. **Contínuas** quando resultam de um número infinito de valores possíveis que possam ser associados em uma escala contínua. São resultantes de ***medições***
    - Valores de criptomoedas
    - Altura de uma pessoa (m, cm, in, etc)
    - Peso de alimentos (kgf, gf, dgf)



### Codificação,  Categorização e Binarização